In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from data_processing import DataProcessing

In [1]:
import mlflow

# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow for Fraud detection")

2024/06/22 09:20:27 INFO mlflow.tracking.fluent: Experiment with name 'MLflow for Fraud detection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/410426737290025736', creation_time=1719037227525, experiment_id='410426737290025736', last_update_time=1719037227525, lifecycle_stage='active', name='MLflow for Fraud detection', tags={}>

In [4]:
# Load the datasets
fraud_data_processor = DataProcessing('../data/raw/Fraud_Data.csv')
fraud_data = fraud_data_processor.preprocess_data()

ip_data_processor = DataProcessing('../data/raw/IpAddress_to_Country.csv')
ip_data = ip_data_processor.preprocess_data()

creditcard_data_processor = DataProcessing('../data/raw/creditcard.csv')
creditcard_data = creditcard_data_processor.preprocess_data()


# Convert datetime strings to datetime objects
fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])

# Extract useful datetime components
fraud_data['signup_hour'] = fraud_data['signup_time'].dt.hour
fraud_data['signup_day'] = fraud_data['signup_time'].dt.dayofweek
fraud_data['purchase_hour'] = fraud_data['purchase_time'].dt.hour
fraud_data['purchase_day'] = fraud_data['purchase_time'].dt.dayofweek

# Drop the original datetime columns
fraud_data = fraud_data.drop(columns=['signup_time', 'purchase_time'])


In [5]:
# For fraud_data
fraud_X = fraud_data.drop(columns=['class'])
fraud_y = fraud_data['class']
# For creditcard_data
creditcard_X = creditcard_data.drop(columns=['Class'])
creditcard_y = creditcard_data['Class']

# Split fraud_data
fraud_X_train, fraud_X_test, fraud_y_train, fraud_y_test = train_test_split(fraud_X, fraud_y, test_size=0.3, random_state=42)

# Split creditcard_data
creditcard_X_train, creditcard_X_test, creditcard_y_train, creditcard_y_test = train_test_split(creditcard_X, creditcard_y, test_size=0.3, random_state=42)

In [6]:
# Define preprocessing for numerical and categorical features
numeric_features = ['purchase_value', 'age']  # Example numeric features
categorical_features = ['source', 'browser', 'sex', 'signup_hour', 'signup_day', 'purchase_hour', 'purchase_day']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [7]:
# Model Selection
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'MLP': MLPClassifier()
}

# Function to train and evaluate models for fraud data
def train_evaluate_model_fraud(model_name, model, X_train, X_test, y_train, y_test):
    # Create a pipeline with preprocessing and model
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', model)])

    # Train the model
    pipeline.fit(X_train, y_train)

    # Make predictions
    y_pred = pipeline.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Infer model signature
    signature = infer_signature(X_train, y_pred)

    # Log metrics and model with MLflow
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model", model_name)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        mlflow.set_tag("Training Info", "All models for Fraud detection")
        model_info = mlflow.sklearn.log_model(
            sk_model=pipeline,
            artifact_path="fraud_model",
            signature=signature,
            input_example=X_train,
            registered_model_name=f"{model_name}_fraud_detection",
        )

    return {
        'model': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

# Function to train and evaluate models for credit card data
def train_evaluate_model_credit(model_name, model, X_train, X_test, y_train, y_test):
    # Create a pipeline with preprocessing and model
    pipeline = Pipeline(steps=[('classifier', model)])

    # Train the model
    pipeline.fit(X_train, y_train)

    # Make predictions
    y_pred = pipeline.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Infer model signature
    signature = infer_signature(X_train, y_pred)

    # Log metrics and model with MLflow
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model", model_name)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)
        mlflow.set_tag("Training Info", "All models for Credit Card Fraud detection")
        model_info = mlflow.sklearn.log_model(
            sk_model=pipeline,
            artifact_path="creditcard_model",
            signature=signature,
            input_example=X_train,
            registered_model_name=f"{model_name}_creditcard_detection",
        )

    return {
        'model': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

In [8]:
# Train and evaluate models on fraud data
fraud_results = []
for model_name, model in models.items():
    fraud_results.append(train_evaluate_model_fraud(model_name, model, fraud_X_train, fraud_X_test, fraud_y_train, fraud_y_test))

# Train and evaluate models on credit card data
creditcard_results = []
for model_name, model in models.items():
    creditcard_results.append(train_evaluate_model_credit(model_name, model, creditcard_X_train, creditcard_X_test, creditcard_y_train, creditcard_y_test))

# Display results
fraud_results_df = pd.DataFrame(fraud_results)
creditcard_results_df = pd.DataFrame(creditcard_results)

print("Fraud Data Results:\n", fraud_results_df)
print("Credit Card Data Results:\n", creditcard_results_df)

c:\Users\Ted\Desktop\Juju\DEV\fraud_detection_model\venv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ted\Desktop\Juju\DEV\fraud_detection_model\venv\Lib\site-packages\mlflow\types\utils.py:406: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing 

Fraud Data Results:
                  model  accuracy  precision    recall  f1_score
0  Logistic Regression  0.906979   0.000000  0.000000  0.000000
1        Decision Tree  0.909119   0.510299  0.569836  0.538427
2        Random Forest  0.957140   0.995209  0.541854  0.701674
3    Gradient Boosting  0.908457   0.935065  0.017074  0.033535
4                  MLP  0.943663   0.779872  0.549443  0.644686
Credit Card Data Results:
                  model  accuracy  precision    recall  f1_score
0  Logistic Regression  0.998666   0.577465  0.602941  0.589928
1        Decision Tree  0.999204   0.732877  0.786765  0.758865
2        Random Forest  0.999602   0.932203  0.808824  0.866142
3    Gradient Boosting  0.998584   0.894737  0.125000  0.219355
4                  MLP  0.998432   0.625000  0.036765  0.069444


Created version '1' of model 'MLP_creditcard_detection'.


In [10]:
# Load a model for inference
logged_model = 'runs:/48369d081c0c46bcad19a91f6bbabbbd/fraud_model'

# Load model as a PyFuncModel
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Use the loaded model to make predictions
sample_input = fraud_X_test.iloc[:5]
predictions = loaded_model.predict(sample_input)
print("Predictions:\n", predictions)

Predictions:
 [0 0 0 0 0]


In [11]:
# Making Prediction by using the loaded model for Fraud data
import mlflow
import pandas as pd

models_runs = [
    ('fraud_model', '48369d081c0c46bcad19a91f6bbabbbd'),
    ('fraud_model', '72bc2c38fb3a49969ee33116b5e00a84'),
    ('fraud_model', '8f48e744470244b5864b6bf373540bd7'),
    ('fraud_model', 'f5a0279ac66d44ffa206aef96fbbb74c'),
    ('fraud_model', '7b35a9c50fca4ade979c654d35f784d7')
]
sample_input = fraud_X_test.iloc[:5]

results_dfs = {}

for model_name, run_id in models_runs:
    logged_model = f'runs:/{run_id}/{model_name}'
    
    # Load the model as a PyFuncModel
    loaded_model = mlflow.pyfunc.load_model(logged_model)
    
    # Make predictions
    predictions = loaded_model.predict(sample_input)

    # Create a DataFrame for the predictions
    results_df = pd.DataFrame({
        'Model': model_name,
        'Run ID': run_id,
        'Index': range(len(predictions)),
        'Prediction': predictions
    })
    
    # Store the DataFrame in the dictionary
    results_dfs[f"{model_name}_{run_id}"] = results_df
    
# Display each DataFrame
for key, df in results_dfs.items():
    print(f"Predictions for {key}:\n{df}\n")


Predictions for fraud_model_48369d081c0c46bcad19a91f6bbabbbd:
         Model                            Run ID  Index  Prediction
0  fraud_model  48369d081c0c46bcad19a91f6bbabbbd      0           0
1  fraud_model  48369d081c0c46bcad19a91f6bbabbbd      1           0
2  fraud_model  48369d081c0c46bcad19a91f6bbabbbd      2           0
3  fraud_model  48369d081c0c46bcad19a91f6bbabbbd      3           0
4  fraud_model  48369d081c0c46bcad19a91f6bbabbbd      4           0

Predictions for fraud_model_72bc2c38fb3a49969ee33116b5e00a84:
         Model                            Run ID  Index  Prediction
0  fraud_model  72bc2c38fb3a49969ee33116b5e00a84      0           0
1  fraud_model  72bc2c38fb3a49969ee33116b5e00a84      1           0
2  fraud_model  72bc2c38fb3a49969ee33116b5e00a84      2           0
3  fraud_model  72bc2c38fb3a49969ee33116b5e00a84      3           0
4  fraud_model  72bc2c38fb3a49969ee33116b5e00a84      4           0

Predictions for fraud_model_8f48e744470244b5864b6bf373540b

In [12]:
# Making Prediction by using the loaded model for Credit card data
import mlflow
import pandas as pd

models_runs = [
    ('creditcard_model', 'a4f2a14a3a9741e993ff861892ed210d'),
    ('creditcard_model', '6aed1e8ba5d648359b95a9586fff8053'),
    ('creditcard_model', '5aaec3c223f64856b6afa0cd52305551'),
    ('creditcard_model', '344bc4ef2cf24ad49e923e56eb68b796'),
    ('creditcard_model', '8ace3b2744db4d6cb597278d90f717c8')
]
sample_input = creditcard_X_test.iloc[:5]

results_dfs = {}

for model_name, run_id in models_runs:
    logged_model = f'runs:/{run_id}/{model_name}'
    
    # Load the model as a PyFuncModel
    loaded_model = mlflow.pyfunc.load_model(logged_model)
    
    # Make predictions
    predictions = loaded_model.predict(sample_input)

    # Create a DataFrame for the predictions
    results_df = pd.DataFrame({
        'Model': model_name,
        'Run ID': run_id,
        'Index': range(len(predictions)),
        'Prediction': predictions
    })
    
    # Store the DataFrame in the dictionary
    results_dfs[f"{model_name}_{run_id}"] = results_df
    
# Display each DataFrame
for key, df in results_dfs.items():
    print(f"Predictions for {key}:\n{df}\n")

Predictions for creditcard_model_a4f2a14a3a9741e993ff861892ed210d:
              Model                            Run ID  Index  Prediction
0  creditcard_model  a4f2a14a3a9741e993ff861892ed210d      0           1
1  creditcard_model  a4f2a14a3a9741e993ff861892ed210d      1           0
2  creditcard_model  a4f2a14a3a9741e993ff861892ed210d      2           0
3  creditcard_model  a4f2a14a3a9741e993ff861892ed210d      3           0
4  creditcard_model  a4f2a14a3a9741e993ff861892ed210d      4           0

Predictions for creditcard_model_6aed1e8ba5d648359b95a9586fff8053:
              Model                            Run ID  Index  Prediction
0  creditcard_model  6aed1e8ba5d648359b95a9586fff8053      0           1
1  creditcard_model  6aed1e8ba5d648359b95a9586fff8053      1           0
2  creditcard_model  6aed1e8ba5d648359b95a9586fff8053      2           0
3  creditcard_model  6aed1e8ba5d648359b95a9586fff8053      3           0
4  creditcard_model  6aed1e8ba5d648359b95a9586fff8053      4  